In [1]:
from typing import Callable, List
import random

import numpy as np
import pandas as pd
import igraph as ig
import leidenalg as la
import matplotlib.pyplot as plt

In [2]:
# Wczytywanie danych
df = pd.read_csv("DaneSiec.csv", skiprows=3)
df

,<AS_nr>,<pref>,<1st_s>,<s_nr>,<ld_nr>,<del>,<jit>,<los>,<hop>
0,1,256,1,85,3356,39,47,0.0003,1
1,0,0,0,0,3705,23,10,0.0003,1
2,2,65536,86,85,34,21,7,0.0009,1
3,0,0,0,0,29076,49,37,0.0002,1
4,0,0,0,0,42226,33,46,0.0002,1
...,...,...,...,...,...,...,...,...,...
206969,393239,512,4191652,85,209,26,34,0.0008,1
206970,438802,5120,4191737,85,43802,10,49,0.0002,1
206971,12845938,256,4191822,85,12741,24,46,0.0004,1
206972,12845948,2048,4191907,85,20959,45,34,0.0001,1


In [3]:
def data_transform(df: pd.DataFrame) -> ig.Graph:
    df.drop((df[df["<ld_nr>"] == 0]).index, inplace=True)
    df.replace({"<AS_nr>": {0: np.nan}}, inplace=True)
    df["<AS_nr>"] = df["<AS_nr>"].ffill(axis=0).astype(int)
    df.drop_duplicates(inplace=True)
    nodes = set(df["<AS_nr>"].values).union(set(df["<ld_nr>"].values))
    labels = dict(zip(nodes, np.arange(len(nodes))))
    edges = []
    weights = []
    for row in df.itertuples():
        start = labels[row[1]]
        end = labels[row[5]]
        weight = float(row[6])
        edges.append((start, end))
        weights.append(weight)
    return ig.Graph(n=len(nodes), edges=edges, edge_attrs={"weight": weights}, directed=True), labels

In [4]:
graph, labels = data_transform(df)

In [5]:
# Nazwy node'ów, ale przelabelowane

all_nodes = list(labels.values())

In [6]:
x1, x2 = random.choice(all_nodes), random.choice(all_nodes)
print(f"Finding path from {x1} to {x2}")

Finding path from 13494 to 13279


In [7]:
graph.get_shortest_path(x1, to=x2)

[13494, 2297, 155, 424, 13279]

In [8]:
# 10000 losowych
N = 1000
for _ in range(N):
    x1, x2 = random.choice(all_nodes), random.choice(all_nodes)
    graph.get_shortest_path(x1, to=x2, weights="weight")

In [9]:
# 10 losowych
N = 10
for _ in range(N):
    x1, x2 = random.choice(all_nodes), random.choice(all_nodes)
    graph.get_all_shortest_paths(x1, to=x2, weights="weight")

In [10]:
graph.get_all_shortest_paths(x1, to=x2)

[[18753, 424, 12427, 18605]]

In [ ]:
# Leiden Algorithm

partition = la.find_partition(
    graph=graph,
    partition_type=la.ModularityVertexPartition,
    n_iterations=100,
    weights="weight",
)

hierarchy = partition.aggregate_partition()
membership = partition.membership

def transform_hierarchy(graph: ig.Graph, membership: np.ndarray, hierarchy: la.VertexPartition.ModularityVertexPartition) -> None:
    l = len(np.unique(membership))
    distances = np.zeros((l, l), dtype=float)
    for edge in graph.es:
        src, tgt = membership[edge.source], membership[edge.target]
        w = edge["weight"]
        distances[src][tgt] = min(distances[src][tgt], w) if distances[src][tgt] > 0.0 else w
    hierarchy.graph.simplify(
        loops=True,
        multiple=False,
        combine_edges=False
    )
    for edge in hierarchy.graph.es:
        edge["weight"] = distances[edge.source][edge.target]


transform_hierarchy(graph, membership, hierarchy)

In [12]:
for e in hierarchy.graph.es:
    print(e.source, e.target, e["weight"])

0 1 5.0
0 2 5.0
0 3 10.0
0 4 5.0
0 14 5.0
0 13 6.0
0 5 5.0
0 10 5.0
0 27 6.0
0 8 5.0
0 16 6.0
0 17 5.0
0 15 7.0
0 11 5.0
0 24 12.0
0 12 6.0
0 22 18.0
0 29 5.0
0 33 14.0
0 6 6.0
0 34 8.0
0 7 10.0
0 25 6.0
0 9 7.0
0 26 7.0
0 19 9.0
0 18 5.0
0 45 10.0
0 23 11.0
0 20 26.0
0 37 25.0
1 0 5.0
1 14 5.0
1 2 5.0
1 4 5.0
1 3 5.0
1 29 11.0
1 5 5.0
1 17 5.0
1 12 5.0
1 8 5.0
1 16 5.0
1 11 5.0
1 10 5.0
1 19 7.0
1 6 5.0
1 22 6.0
1 15 5.0
1 7 5.0
1 13 5.0
1 21 5.0
1 18 8.0
1 32 29.0
1 25 6.0
1 26 5.0
1 27 7.0
1 30 26.0
1 9 5.0
1 20 8.0
1 23 27.0
1 33 20.0
1 38 12.0
1 31 13.0
1 39 7.0
2 14 5.0
2 0 5.0
2 4 5.0
2 5 5.0
2 13 5.0
2 22 5.0
2 3 5.0
2 1 5.0
2 11 5.0
2 8 5.0
2 10 5.0
2 7 5.0
2 12 5.0
2 25 5.0
2 20 5.0
2 6 5.0
2 16 5.0
2 17 5.0
2 15 5.0
2 21 5.0
2 19 5.0
2 9 5.0
2 26 5.0
2 32 6.0
2 30 9.0
2 23 6.0
2 18 5.0
2 42 13.0
2 27 30.0
2 37 36.0
2 33 30.0
2 28 23.0
2 29 44.0
2 43 13.0
2 44 7.0
2 35 22.0
2 40 38.0
2 41 23.0
2 36 25.0
2 38 13.0
3 1 5.0
3 2 5.0
3 6 5.0
3 16 5.0
3 4 7.0
3 20 6.0
3 5 5.0
3 8 6

In [13]:
def generate_heurestic(membership: np.ndarray, hierarchy: la.VertexPartition.ModularityVertexPartition) -> Callable[[ig.Graph, int, int], float]:
    n = len(np.unique(membership))
    community_distances = np.zeros((n, n), dtype=float)
    for i in range(n):
        for j in range(n):
            community_distances[i][j] = hierarchy.graph.distances(i, j, weights="weight")[0][0]
    def heurestic(graph: ig.Graph, src: int, tgt: int) -> float:
        dist = community_distances[membership[src]][membership[tgt]]
        return dist
    return heurestic 

heurestic = generate_heurestic(membership, hierarchy)

In [14]:
x1, x2 = random.choice(all_nodes), random.choice(all_nodes)

In [15]:
graph.get_shortest_path_astar(0, to=1, weights="weight", heuristics=heurestic)

[0, 1386, 2364, 899, 1171, 17909, 1]

In [16]:
graph.get_shortest_path(0, to=1, weights="weight")

[0, 1386, 2364, 899, 1171, 17909, 1]

In [19]:
N = 100
for _ in range(N):
    x1, x2 = random.choice(all_nodes), random.choice(all_nodes)
    graph.get_shortest_path_astar(x1, to=x2, weights="weight", heuristics=heurestic)

In [20]:
N = 100
for _ in range(N):
    x1, x2 = random.choice(all_nodes), random.choice(all_nodes)
    graph.get_shortest_path(x1, to=x2, weights="weight")